In [3]:
import pandas as pd
import numpy as np


In [4]:
a = {
    "buildship": "https://github.com/eclipse/steady.git",
    "eclips-collections": "https://github.com/eclipse/eclipse-collections.git",
    "hawkbit": "https://github.com/eclipse/hawkbit.git",
    "jkube": "https://github.com/eclipse/jkube.git",
    "kura": "https://github.com/eclipse/kura.git",
    "jifa": "https://github.com/eclipse/microprofile.git",
    "milo": "https://github.com/eclipse/milo.git",
    "openvsx": "https://github.com/eclipse/openvsx.git",
    "steady": "https://github.com/eclipse/steady.git",
    "xtext": "https://github.com/eclipse/xtext.git"
}

In [7]:
# Step 3: Adjust the 'category' field based on conditions
def adjust_category(row):
    if pd.notna(row['category_df1']) and row['category_df1'] == "DOS":
        return "DOS" if pd.isna(row['category_df2']) else f"DOS,{row['category_df2']}"
    return row['category_df2'] if pd.notna(row['category_df2']) else ""


def merge_dataframes(df1, df2):
    # Step 1: Rename 'label' in df1 to 'category' for uniformity and drop 'label' from df1 if it exists
    df1 = df1.rename(columns={'label': 'category'})
    df1['category'] = df1['category'].apply(lambda x: "DOS" if x == "positive" else "")

    # Step 2: Merge the DataFrames on specified columns
    merged_df = pd.merge(df1, df2, 
                        on=["content", "method_name", "parameters", "access_modifier"], 
                        how='outer', 
                        suffixes=('_df1', '_df2'))



    

MODEl Implementation (Multi label model)

In [2]:
import ktrain
import pandas as pd
from ktrain import tabular
import numpy as np
import pandas   as pd
from sklearn.utils import shuffle
from sklearn.metrics import matthews_corrcoef, roc_auc_score
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt

In [3]:
def get_model(a,project):
    # load the model
    df = pd.read_csv(f'../data/{project}_final.csv')
    # drop all rows in df
    df = df.drop(df.index)
    print(df.shape)
    for p,u in a.items():
        if p == project:
            continue
        df1 = pd.read_csv(f'../data/{p}_final.csv')
        df = pd.concat([df,df1])
    df = df.drop(columns=['comments','content'],axis=1)
    # Count the number of rows in the "not_smell" class
    # not_smell_count = (df['catagory'] == 'not_smell').sum()

    # # Calculate the number of rows to drop (one-third of the count)
    # rows_to_drop = not_smell_count // 3

    # # Filter the DataFrame to exclude one-third of the "not_smell" class
    # filtered_smell = df[df['catagory'] == 'not_smeel'].sample(frac=(1/3))  # Selecting 2/3 of 'not_smell' instances
    # # print(filtered_smell.shape)
    
    # defalut_smell = df[df['catagory'] != 'not_smeel']
    # print(defalut_smell.shape)
    # df = pd.concat([filtered_smell, defalut_smell])
    
    train_df = shuffle(df)
    test_df = pd.read_csv(f'../data/{project}_final.csv')
    test_df = test_df.drop(columns=['comments','content'],axis=1)
    # print(df['catagory'].value_counts())
    
    # df = df.sample(frac=1).reset_index(drop=True)
    
    # np.random.seed(42)
    # p = 0.1 # 10% for test set
    # prop = 1-p
    # temp_df = df.copy()
    # msk = np.random.rand(len(temp_df)) < prop
    # train_df = temp_df[msk]
    # test_df = temp_df[~msk]
    # print(train_df["label"].value_counts())
    # print(test_df["label"].value_counts())
    test_df['index'] = test_df.index
    train_df['index'] = train_df.index  
    return train_df, test_df

In [4]:
def apply_threshold(probabilities, threshold=0.28):
    return (probabilities >= threshold).astype(int)

In [5]:
def get_lean(train_df,test_df):
    print(test_df.iloc[1])
    trn,val,preproc = tabular.tabular_from_df(train_df, label_columns=['is','set','get','DOS'], random_state=42)
    model  = tabular.tabular_classifier(name='mlp',multilabel=True,train_data=trn,hidden_layers=[1000,1000,500],hidden_dropouts=[0.2,0.5,0.5])
    learner = ktrain.get_learner(model,train_data=trn,val_data=val,batch_size=32)
    # learner.lr_find(show_plot=True, max_epochs=5)
    data = {"get":0,"is":1,"not_LA":2,"set":3}
    print(preproc.get_classes())
    learner.fit_onecycle(5e-3, 1)
    print(learner.evaluate(val, class_names=preproc.get_classes()))
    predictor = ktrain.get_predictor(learner.model, preproc)

    
    preds = predictor.predict(test_df, return_proba=True)
    
    # try:
    #     # Compute ROC curve and ROC area for each class
    #     fpr = dict()
    #     tpr = dict()
    #     roc_auc = dict()
    #     for class_name, class_idx in data.items():
    #         y_true_class = (test_df['category'] == class_name).astype(int)
    #         y_score_class = preds[:, class_idx]
    #         fpr[class_name], tpr[class_name], _ = roc_curve(y_true_class, y_score_class)
    #         roc_auc[class_name] = auc(fpr[class_name], tpr[class_name])
            
            
    #     # Plot ROC curve for each class
    #     plt.figure()
    #     for class_name, class_idx in data.items():
    #         plt.plot(fpr[class_name], tpr[class_name], label=f'ROC curve (AUC = {roc_auc[class_name]:0.2f}) for {class_name}')
    #     plt.plot([0, 1], [0, 1], 'k--')
    #     plt.xlim([0.0, 1.0])
    #     plt.ylim([0.0, 1.05])
    #     plt.xlabel('False Positive Rate')
    #     plt.ylabel('True Positive Rate')
    #     plt.title('Receiver Operating Characteristic (ROC) Curve')
    #     plt.legend(loc="lower right")
    #     plt.show()
    #     # Calculate AUC for each class

    #     auc_scores = []
    #     for class_name, class_idx in data.items():  # Iterate over each class
    #         y_true_class = (test_df['category'] == class_name).astype(int)  # Convert true labels to binary for the current class
    #         y_pred_class = preds[:, class_idx]  # Predicted probabilities for the current class
            
    #         # print(f'Class {class_name}:')
    #         # print('y_true_class:', y_true_class)
    #         # print('y_pred_class:', y_pred_class)
            
    #         auc_score = roc_auc_score(y_true_class, y_pred_class)  # Compute ROC AUC
    #         auc_scores.append(auc_score)
    #         # print(f'AUC for class {class_name}: {auc_score}')

    #     # Overall AUC can also be computed by averaging the AUC scores for each class
    #     overall_auc = np.mean(auc_scores)
    #     print(f'Overall AUC: {overall_auc}')
    #     # Calculate MCC for each class
    #     mcc_scores = []
    #     for class_name, class_idx in data.items():  # Iterate over each class
    #         y_true_class = (test_df['category'] == class_name).astype(int)  # True labels for the current class
    #         y_pred_class = np.argmax(preds, axis=1)  # Predicted labels for the current class
            
    #         # print(f'Class {class_name}:')
    #         # print('y_true_class:', y_true_class.shape)
    #         # print('y_pred_class:', y_pred_class.shape)
            
    #         # Filter y_pred_class to match the same samples as y_true_class
    #         y_pred_class_filtered = y_pred_class[test_df['category'] == class_name]
    #         # print(f"y_pred_class_filtered: {y_pred_class_filtered.shape}")
    #         mcc_score = matthews_corrcoef(y_true_class, y_pred_class)  # Compute MCC
    #         mcc_scores.append(mcc_score)
    #         print(f'MCC for class {class_name}: {mcc_score}')

    #     # Overall MCC can also be computed by averaging the MCC scores for each class
    #     overall_mcc = np.mean(mcc_scores)
    #     print(f'Overall MCC: {overall_mcc}')
    #     print(test_df.iloc[2])
    #     data = {"get":0,"is":1,"not_LA":2,"set":3}
    #     print("predict output is here")
    #     pred = predictor.predict(test_df[1:2])
    #     print(test_df.iloc[1])
    #     print(f"the prediction is {pred[0]} this")
    #     print()
    #     print()
    #     predictor.explain(test_df,row_num=1,class_id=data[pred[0]])
    # except ValueError:
    #     print(f'ValueError {ValueError}')
    
    return predictor

In [19]:
# # Function to encode categories, with handling for NaN or empty strings
# def encode_categories(row, categories):
#     if pd.isna(row) or row == "":
#         return pd.Series({category: 0 for category in categories})
#     encoded = {category: 0 for category in categories}
#     categories_in_row = str(row).split(',')  # Ensure conversion to string to handle NaNs gracefully
#     for category in categories_in_row:
#         if category in encoded:
#             encoded[category] = 1
#     return pd.Series(encoded)


In [20]:
# classes = ['is','set','get','DOS']
# for project,url in a.items():
#     df = pd.read_csv(f'../data/{project}_final.csv')
#     encoded_categories = df['category'].apply(encode_categories, categories=classes)
#     df = pd.concat([df.drop('category', axis=1), encoded_categories], axis=1)
#     df.to_csv(f'../data/{project}_final.csv', index=False)

    

In [6]:
def model_save(predictor,test_df,output_file):
    print()
    print()
    model_path = f'model/{output_file}_features_model'
    print(model_path)
    print()
    print()
    predictor.save(model_path)

In [28]:
!git status

Enumerating objects: 187, done.
Counting objects: 100% (187/187), done.
Delta compression using up to 8 threads
Compressing objects: 100% (156/156), done.
error: RPC failed; HTTP 400 curl 92 HTTP/2 stream 0 was not closed cleanly: CANCEL (err 8)
send-pack: unexpected disconnect while reading sideband packet
Writing objects: 100% (176/176), 12.42 GiB | 49.95 MiB/s, done.
Total 176 (delta 80), reused 19 (delta 0), pack-reused 0
fatal: the remote end hung up unexpectedly
Everything up-to-date


In [29]:
!git add .

In [30]:
!git commit -m "multi-label classification model with explaination"

[main 44a3be7] multi-label classification model with explaination
 90 files changed, 176322 insertions(+), 173268 deletions(-)
 create mode 100644 code/gpt_labeling.ipynb
 create mode 100644 code/merge_content.ipynb
 create mode 100644 code/merge_features.ipynb
 create mode 100644 code/milo1_data.csv
 create mode 100644 code/milo2_data.csv
 create mode 100644 code/milo3_data.csv
 create mode 100644 code/milo4_data.csv
 create mode 100644 code/milo5_data.csv
 delete mode 100644 code/model/buildship_combined_model/tf_model.h5
 delete mode 100644 code/model/buildship_combined_model/tf_model.preproc
 delete mode 100644 code/model/buildship_content_model/tf_model.h5
 delete mode 100644 code/model/buildship_content_model/tf_model.preproc
 delete mode 100644 code/model/buildship_features_model/tf_model.h5
 delete mode 100644 code/model/buildship_features_model/tf_model.preproc
 delete mode 100644 code/model/buildship_multi_label_model/tf_model.h5
 delete mode 100644 code/model/buildship_multi

In [7]:
a = {
    "buildship": " ", 
    "eclips-collections": " ",
    "jifa": " ",
    "jkube": " ",
    "hawkbit": "https://github.com/eclipse/hawkbit.git",
    "kura": "https://github.com/eclipse/kura.git",
    "milo": "https://github.com/eclipse/milo.git",
    "openvsx": "https://github.com/eclipse/openvsx.git",
    "steady": "https://github.com/eclipse/steady.git",
    "xtext": "https://github.com/eclipse/xtext.git"
}
for project,url in a.items():
    print(f"the project is {project}")  
    train_df,test_df = get_model(a,project)
    predictor = get_lean(train_df,test_df)
    # model_save(predictor,test_df,project)

the project is buildship
(0, 11)
access_modifier                          static
method_name                      getArtifactUri
parameters         String g, String a, String v
return_type                                void
returns                              return s; 
is                                            0
set                                           0
get                                           1
DOS                                           0
index                                         1
Name: 1, dtype: object
processing train: 130271 rows x 10 columns
['is', 'set', 'get', 'DOS']
       is  set  get  DOS
33158   1    0    0    0
1218    0    0    0    0
2658    0    0    0    1
88091   1    0    0    0
29450   0    0    0    0
processing test: 14267 rows x 10 columns
['is', 'set', 'get', 'DOS']
       is  set  get  DOS
87262   1    0    0    0
41057   1    0    0    0
78228   1    0    0    0
71522   0    0    0    0
83085   1    0    0    0
Is Multi-Label? True
don

/Users/devang/miniconda3/envs/flask/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/devang/miniconda3/envs/flask/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/devang/miniconda3/envs/flask/lib/python3.11/site-packages/ktrain/core.py:226: UserWarning: Confusion matrices do not currently support multilabel classification, so returning None
  warnings.warn(


the project is eclips-collections
(0, 11)
access_modifier                                               public
method_name                                                   equals
parameters                                                  Object o
return_type                                                  boolean
returns            return true; return false; return Arrays.equal...
is                                                                 0
set                                                                0
get                                                                0
DOS                                                                1
index                                                              1
Name: 1, dtype: object
processing train: 127966 rows x 10 columns
['is', 'set', 'get', 'DOS']
       is  set  get  DOS
3373    0    0    1    0
22467   0    0    0    0
4435    0    0    0    0
66442   0    0    0    0
16426   0    0    1    0
processing test: 13982 r

/Users/devang/miniconda3/envs/flask/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/devang/miniconda3/envs/flask/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/devang/miniconda3/envs/flask/lib/python3.11/site-packages/ktrain/core.py:226: UserWarning: Confusion matrices do not currently support multilabel classification, so returning None
  warnings.warn(


              precision    recall  f1-score   support

          is       0.86      0.73      0.79      2011
         set       0.98      0.90      0.94       947
         get       0.51      0.92      0.65      3013
         DOS       0.94      0.57      0.71      1862

   micro avg       0.67      0.78      0.72      7833
   macro avg       0.82      0.78      0.77      7833
weighted avg       0.76      0.78      0.74      7833
 samples avg       0.41      0.41      0.41      7833

None
['is', 'set', 'get', 'DOS']
   is  set  get  DOS
0   0    0    0    1
1   0    0    0    1
2   0    0    0    0
3   0    0    0    1
4   0    0    0    0
the project is jifa
(0, 11)
access_modifier                                               public
method_name                                              deserialize
parameters         JsonElement json, Type typeOfT, JsonDeserializ...
return_type                                            LocalDateTime
returns            return LocalDateTime.parse(js

...

              precision    recall  f1-score   support

          is       0.84      0.80      0.82      2065
         set       0.99      0.93      0.96       900
         get       0.52      0.89      0.66      3182
         DOS       0.93      0.54      0.68      2086

   micro avg       0.68      0.78      0.73      8233
   macro avg       0.82      0.79      0.78      8233
weighted avg       0.76      0.78      0.74      8233
 samples avg       0.42      0.42      0.42      8233

None
['is', 'set', 'get', 'DOS']
   is  set  get  DOS
0   0    0    0    1
1   0    0    0    0
2   0    0    0    1
3   0    0    0    1
4   0    0    0    1
the project is steady
(0, 11)
access_modifier                          static
method_name                      getArtifactUri
parameters         String g, String a, String v
return_type                              String
returns                              return s; 
is                                            0
set                               

/Users/devang/miniconda3/envs/flask/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/devang/miniconda3/envs/flask/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/devang/miniconda3/envs/flask/lib/python3.11/site-packages/ktrain/core.py:226: UserWarning: Confusion matrices do not currently support multilabel classification, so returning None
  warnings.warn(


the project is xtext
(0, 11)
access_modifier                                               public
method_name                                          overrideEditors
parameters         String fileName, IContentType contentType, IEd...
return_type                                      IEditorDescriptor[]
returns            return (IEditorDescriptor[]) result.toArray(ne...
is                                                                 0
set                                                                0
get                                                                0
DOS                                                                1
index                                                              1
Name: 1, dtype: object
processing train: 52702 rows x 10 columns
['is', 'set', 'get', 'DOS']
       is  set  get  DOS
5563    0    1    0    0
15877   0    0    0    0
3062    0    0    0    1
10216   0    1    0    0
2864    0    0    0    0
processing test: 5803 rows x 10 column

/Users/devang/miniconda3/envs/flask/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/devang/miniconda3/envs/flask/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/devang/miniconda3/envs/flask/lib/python3.11/site-packages/ktrain/core.py:226: UserWarning: Confusion matrices do not currently support multilabel classification, so returning None
  warnings.warn(


['is', 'set', 'get', 'DOS']
   is  set  get  DOS
0   0    0    0    1
1   0    0    0    1
2   0    0    0    1
3   0    0    0    1
4   0    0    0    1
